In [13]:
arpabet_to_ipa={
    '<pad>': '<pad>',
    '<unk>': '<unk>',
    '<s>': '<s>',
    '</s>': '</s>',
    'AA0': 'ɑ',
    'AA1': 'ɑ',#ː
    'AA2': 'ɑ',
    'AE0': 'æ',
    'AE1': 'æ',
    'AE2': 'æ',
    'AH0': 'ə',
    'AH1': 'ʌ',
    'AH2': 'ʌ',
    'AO0': 'ɔ',
    'AO1': 'ɔ',#ː
    'AO2': 'ɔ',
    'AW0': 'aʊ',
    'AW1': 'aʊ',
    'AW2': 'aʊ',
    'AY0': 'aɪ',
    'AY1': 'aɪ',
    'AY2': 'aɪ',
    'B': 'b',
    'CH': 'tʃ',
    'D': 'd',
    'DH': 'ð',
    'EH0': 'ɛ',
    'EH1': 'ɛ',
    'EH2': 'ɛ',
    'ER0': 'ɚ',
    'ER1': 'ɚ',
    'ER2': 'ɚ',
    'EY0': 'eɪ',
    'EY1': 'eɪ',
    'EY2': 'eɪ',
    'F': 'f',
    'G': 'ɡ',
    'HH': 'h',
    'IH0': 'ɪ',
    'IH1': 'ɪ',
    'IH2': 'ɪ',
    'IY0': 'i',
    'IY1': 'i',#ː
    'IY2': 'i',
    'JH': 'dʒ',
    'K': 'k',
    'L': 'l',
    'M': 'm',
    'N': 'n',
    'NG': 'ŋ',
    'OW0': 'oʊ',
    'OW1': 'oʊ',
    'OW2': 'oʊ',
    'OY0': 'ɔɪ',
    'OY1': 'ɔɪ',
    'OY2': 'ɔɪ',
    'P': 'p',
    'R': 'ɹ',
    'S': 's',
    'SH': 'ʃ',
    'T': 't',
    'TH': 'θ',
    'UH0': 'ʊ',
    'UH1': 'ʊ',
    'UH2': 'ʊ',
#    'UW': 'uː',
    'UW0': 'u',
    'UW1': 'u',#ː
    'UW2': 'u',
    'V': 'v',
    'W': 'w',
    'Y': 'j',
    'Z': 'z',
    'ZH': 'ʒ',
}

In [421]:
import librosa
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import textgrid
import os
import numpy as np
from transformers import AutoProcessor, AutoModelForCTC
from phonemizer.backend.espeak.wrapper import EspeakWrapper
_ESPEAK_LIBRARY = r"C:\Program Files\eSpeak NG\libespeak-ng.dll"
EspeakWrapper.set_library(_ESPEAK_LIBRARY)
processor_P = AutoProcessor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model_P = AutoModelForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\Alex\anaconda3\envs\BayesPCN\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights 

In [58]:
english_phonemes = ['<pad>', '<unk>', '<s>', '</s>', 'ɑ', 'æ', 'ə', 'ʌ', 'ɔ', 'aʊ', 'aɪ', 'b', 'tʃ', 'd', 'ð', 'ɛ', 'ɚ', 'eɪ', 'f',
 'ɡ', 'h', 'ɪ', 'i', 'dʒ', 'k', 'l', 'm', 'n', 'ŋ', 'oʊ', 'ɔɪ', 'p', 'ɹ', 's', 'ʃ', 't', 'θ', 'ʊ', 'u', 'v', 'w',
 'j', 'z', 'ʒ']#['<pad>', '<s>', '</s>', '<unk>','p', 'b', 't', 'd', 'k', 'ɡ','m', 'n', 'ŋ', 'f', 'v', 'θ', 'ð', 's', 'z', 'ʃ', 'h', 'tʃ', 'dʒ', 'l', 'ɹ', 'w', 'j',"i","ɪ","ʊ","u","e","ɜ","æ","ʌ","ɑ","ɒ","eɪ","ɔɪ","oʊ","aɪ","aʊ"]
english_phoneme_dict = {k: v for k, v in processor_P.tokenizer.get_vocab().items() if k in english_phonemes}

In [2]:
human_result_path=r"..\data\test.xlsx"
human_result = pd.read_excel(human_result_path)
human_result_1a=human_result[human_result["Experiment"]=="1a"]

In [5]:
talkersets=list(set([j for i in set(human_result_1a["TrainingTalkerID"]) for j in i.split(", ")]))

set1_list=[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16]
set2_list=[17,18,19,20,21,22,24,25,26,27,28,29,30,31,37,40]

def get_training_paths(TrainingTalkerID,all_path):
    path_list=[]
    TalkerID=[]
    for each_ID in TrainingTalkerID.split(", "):
        if each_ID[:3]=="CMN":
            TalkerID.append(f"ALL_{each_ID[-3:]}_M_CMN_ENG_HT1")
        else:
            TalkerID.append(f"ALL_{each_ID[-3:]}_M_ENG_ENG_HT1")
    
    for each_path in TalkerID:
        for i in all_path:
            if each_path in i:
                path_list.append(i)
                break
    return path_list
def get_pathset(paths):
    return [os.path.join(dir, each_file) for dir, mid, files in os.walk(paths) for each_file in files if each_file.endswith(".wav")]


In [81]:
talkersets

['ENG_M_131',
 'CMN_M_016',
 'CMN_M_043',
 'CMN_M_037',
 'ENG_M_070',
 'ENG_M_055',
 'CMN_M_032',
 'CMN_M_021',
 'ENG_M_133',
 'ENG_M_066',
 'CMN_M_035']

In [420]:


def split_audio(audio, start_time, end_time, sr):
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)
    return audio[start_sample:end_sample]

# 假设所有音频文件都在 'audio_files' 文件夹下
#audio_dir = r".\speech_files"
audio_dir =r"..\data\all_other"
audio_dir1 =r".\speech_files"
audio_paths = [os.path.join(audio_dir,f) for f in os.listdir(audio_dir) if f.endswith('HT1.wav')] + [os.path.join(audio_dir1,f) for f in os.listdir(audio_dir1) if f.endswith('HT1.wav')]
label_paths = [os.path.join(audio_dir,f) for f in os.listdir(audio_dir) if f.endswith('HT1.TextGrid')] +[os.path.join(audio_dir1,f) for f in os.listdir(audio_dir1) if f.endswith('HT1.TextGrid')]

# 假设每个音频文件对应一个CSV标注文件
audio_clips = []
labels = []
finetunedata = {
    "audio": [],
    "transcription": []
}
set_list=set1_list+set2_list
for audio_path,label_path in zip(audio_paths, label_paths):
    # 加载音频文件
    #audio_path = os.path.join(audio_dir, audio_file)
    audio, sr = librosa.load(audio_path)
    wave_res = librosa.resample(audio, orig_sr=sr, target_sr=16000)

    # 加载标注文件
    #label_path = os.path.join(audio_dir, label_file)
    tg = textgrid.TextGrid.fromFile(label_path)
    tg_sentence = tg[0]
    for _,i in enumerate(tg[0]):
        if i.mark!="":
            tg_sentence[_-1].maxTime=tg_sentence[_].minTime
    tg_sentence = [i for i in tg_sentence if i.mark!=""]
    #tg_sentence = [each for _,each in enumerate(tg_sentence) if _ in set_list]


    # 切割音频文件
    for each_sentence in tg_sentence:
        start_time = each_sentence.minTime
        end_time = each_sentence.maxTime
        marks=[]
        clip = split_audio(wave_res, start_time, end_time, 16000)
        #ipalist=[arpabet_to_ipa[i.mark] for i in tg[-1] if tg_sentence[0].minTime<=i.minTime and tg_sentence[0].maxTime>=i.maxTime and i.mark!="" and i.mark!="sp" and i.mark!="sil"]
        #audio_clips.append(clip)
        finetunedata["audio"].append(clip)
        #labels.append(" ".join(ipalist))#np.array([[english_phoneme_dict[i] for i in ipalist]])
        finetunedata["transcription"].append(each_sentence.mark)#[english_phoneme_dict[i] for i in ipalist])


In [349]:
set(finetunedata["transcription"])

{'A BOY FELL FROM A WINDOW',
 'A BOY FELL FROM THE WINDOW',
 'A LETTER FELL ON THE FLOOR',
 'A TOWEL IS NEAR THE SINK',
 'BIG DOGS CAN BE DANGEROUS',
 'FATHER FORGOT THE BREAD',
 'FLOWERS CAN GROW IN THE POT',
 'HE CLIMBED UP THE LADDER',
 'HE GREW LOTS OF VEGETABLES',
 'HE HUNG UP HIS RAINCOAT',
 "HE'S SKATING WITH HIS FRIEND",
 "HE'S WASHING HIS FACE WITH SOAP",
 'HER COAT IS ON A CHAIR',
 "IT'S TIME TO GO TO BED",
 'MILK COMES IN A CARTON',
 'MOTHER CUT THE BIRTHDAY CAKE',
 'MOTHER GOT A SAUCEPAN',
 'MOTHER READ THE INSTRUCTIONS',
 'RAIN IS GOOD FOR TREES',
 'SCHOOL GOT OUT EARLY TODAY',
 'SHE ARGUES WITH HER SISTER',
 'SHE CUT THE STEAK WITH HER KNIFE',
 'SHE FOUND HER PURSE IN THE TRASH',
 'SHE LOST HER CREDIT CARD',
 'SHE STOOD NEAR THE WINDOW',
 'SHE TOOK OFF HER FUR COAT',
 'SHE USES HER SPOON TO EAT',
 "SHE'S DRINKING FROM HER OWN CUP",
 'SOMEBODY STOLE THE MONEY',
 'SOMEONE IS CROSSING THE ROAD',
 'SUGAR IS VERY SWEET',
 'SWIMMERS CAN HOLD THEIR BREATH',
 'THE APPLE PIE IS GO

In [297]:
x=processor_P.tokenizer('ə b oːɹ f ɜː f ɹ ʌ m z ə w ɪ n d oʊ').input_ids

In [330]:
processor_P.tokenizer(finetunedata["transcription"][0]).input_ids

[20, 26, 100, 23, 14, 8, 23, 27, 33, 13, 22, 7, 32, 17, 4, 12, 49]

In [331]:
processor_P.batch_decode(processor_P.tokenizer(finetunedata["transcription"][0]).input_ids)

['ɐ',
 'b',
 'ɔ ɪ',
 'f',
 'ɛ',
 'l',
 'f',
 'ɹ',
 'ʌ',
 'm',
 'ð',
 'ə',
 'w',
 'ɪ',
 'n',
 'd',
 'o ʊ']

In [332]:
def preprocess_function(examples):
    #speech_array, sampling_rate = sf.read(examples["audio_path"])
    examples["input_values"] = processor_P(examples["audio"], sampling_rate=16000).input_values[0]
    examples["labels"] = processor_P.tokenizer(examples["transcription"]).input_ids  #examples["transcription"]

    return examples

In [422]:
from datasets import Dataset
finetunedataset = Dataset.from_dict(finetunedata)
dataset1 = finetunedataset.map(preprocess_function)

Map:   0%|          | 0/2340 [00:00<?, ? examples/s]

In [ ]:

wave, sr = librosa.load(os.path.join(audio_dir,audio_files[0]))
wave_res = librosa.resample(wave, orig_sr=sr, target_sr=16000)
input=processor_P(wave_res, sampling_rate=16000, return_tensors="pt").input_values.to(device)
model_P.to(device)
with torch.no_grad():
    out_encoder=model_P(input).logits

outind=torch.argmax(out_encoder,dim=-1).cpu().numpy()
transcription = processor_P.batch_decode(outind)[0].split(" ")
torch.cuda.empty_cache()

In [339]:
processor_P.batch_decode(outind)[0]

'ə b oːɹ f ɜː f ɹ ʌ m z ə w ɪ n d oʊ z ð ə w aɪ f h p h ɜː h æ z b ə n p ɪ k t ɑː k s k æ n p iː d æ n d ɚ s ð ʃ uː s w ɜː w æ ɹ i d ɜː ɾ i ð ə p l eɪ ɚ l ɔː z ɚ ʃ uː ʌ m m ɚ d i z d oʊ z ə m ʌ n i ð ə f æ w ʌ z w æ ɹ i h ɑːɹ t ʃ ɪ z ŋ k ɪ ŋ f ɹ ʌ m x oʊ n k ɑː p ð ə p ɪ tʃ ɜː k eɪ m f ɹ ʌ m ə p əl k ð ə k ɑːɹ ɪ z k oʊ ɪ ŋ t uː f s t ð p eɪ n t dʒ ɪ p z ŋ z ə k ɹ ŋ d ð ə t ɑːɹ f aʊ ɑː ŋ z ə f l ɔː ð ə f m l i l æ k s f iː ʃ ð ə p n æ n z ɑːɹ t uː ɹ p iː k uː l ɑːɹ s oʊ w ɛ ɹ ɪ t ə b ɔː ʃ iː ɑːɹ ɡ ɚ w ɪ z h ɜː s ɪ s t ɚ ə k tʃ ɪ n w ɪ n d oʊ w ɜː k l n h iː x aʊ ŋ ɑː p ɪ z ɹ ɪ ŋ k ɑːɹ t z ð ə m eɪ m ɪ n b ɹ ɔː t ə l æ t ɚ ð ə m ʌ z ɚ x ɜː z ə b eɪ b i ʃ iː f ɑːɹ ŋ x ɜː p ɜː ɪ n d ɚ ts.h æ ð ə t eɪ b əl h æ z s ɹ iː l æ k s ð ə tʃ ɪ d ɹ ə n w ɪ d æ z ð ɚ t ɹ eɪ n x ɜː k ɑːɹ d ɪ z ɑː ŋ d ɚ t ts.h aɪ ð ə k ɜː ɪ z f ɪ k s ɪ ŋ h ɜː d ɹ æ s ə t aɪ m t uː k oʊ t uː b æ d m ʌ z ɚ ɹ ɪ z ə ɪ n s t ʌ k ʃ ə n s ð ə t ɑː z ɪ t ŋ z ʌ n m ɪ t f ɑː z ɚ f ɔːɹ k ɔː z ə b ɹ æ d ð ə dʒ ɔː d ɡ oʊ z æ p ʌ h 

In [160]:
processor_P.tokenizer('a boy fell from a lost', return_tensors="pt", padding=True).input_ids

tensor([[ 20,  26, 100,  23,  14,   8,  23,  27,  33,  13,  20,   8,  40,   5,
           6]])

In [234]:
import panphon.distance
dst = panphon.distance.Distance()
dst.dolgo_prime_distance(u'ɐ bɔɪ fɛl fɹʌm ðə wɪndoʊ', u'a boy fel from te windoo')


0

In [368]:
from difflib import SequenceMatcher

def compare_phonemes(slow, fast):
    matcher = SequenceMatcher(None, slow.split(), fast.split())
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == "delete":
            print(f"Deleted in fast speech: {slow.split()[i1:i2]}")

slow = "ə b ə v"
fast = "ə b ʌ v"
compare_phonemes(slow, fast)


In [371]:
matcher = SequenceMatcher(None, slow.split(), fast.split())
matcher.get_opcodes()

[('equal', 0, 2, 0, 2), ('replace', 2, 3, 2, 3), ('equal', 3, 4, 3, 4)]

In [248]:
import panphon
ft = panphon.FeatureTable()
x=ft.word_fts(u't')
x[0]

<Segment [-syl, -son, +cons, -cont, -delrel, -lat, -nas, -strid, -voi, -sg, -cg, +ant, +cor, -distr, -lab, -hi, -lo, -back, -round, -velaric, 0tense, -long, 0hitone, 0hireg]>

In [220]:
list(processor_P.batch_decode(processor_P.tokenizer('a boy fell from the window', return_tensors="pt", padding=True).input_ids))

['ɐ b ɔɪ f ɛ l f ɹ ʌ m ð ə w ɪ n d oʊ']

In [38]:

'''tg = textgrid.TextGrid.fromFile(os.path.join(audio_dir, label_files[0]))
tg_sentence = tg[0]
for _,i in enumerate(tg[0]):
    if i.mark!="":
        tg_sentence[_-1].maxTime=tg_sentence[_].minTime
tg_sentence = [i for i in tg_sentence if i.mark!=""]
tg_sentence = [each for _,each in enumerate(tg_sentence) if _ in set_list]'''
            

32

In [374]:
import whisper

# 加载 Whisper 模型
model = whisper.load_model("base")


100%|███████████████████████████████████████| 139M/139M [00:13<00:00, 11.0MiB/s]
c:\Users\Alex\anaconda3\envs\BayesPCN\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues relat

In [378]:
model

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-5): 6 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=512, out_features=512, bias=True)
          (key): Linear(in_features=512, out_features=512, bias=False)
          (value): Linear(in_features=512, out_features=512, bias=True)
          (out): Linear(in_features=512, out_features=512, bias=True)
        )
        (attn_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (mlp_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((512,), eps=1e-05,

In [377]:
model.transcribe(wave_res[start:end])

c:\Users\Alex\anaconda3\envs\BayesPCN\lib\site-packages\whisper\model.py:124: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  a = scaled_dot_product_attention(


{'text': ' Big dots can be dangerous.',
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 2.0,
   'text': ' Big dots can be dangerous.',
   'tokens': [50364, 5429, 15026, 393, 312, 5795, 13, 50464],
   'temperature': 0.0,
   'avg_logprob': -0.5766706466674805,
   'compression_ratio': 0.7647058823529411,
   'no_speech_prob': 0.014791703782975674}],
 'language': 'en'}

In [353]:
from transformers import Wav2Vec2ForCTC, TrainingArguments, Trainer
import jiwer
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred_str = processor_P.batch_decode(pred_ids)
    labels_ids = pred.label_ids
    labels_ids[labels_ids == -100] = processor_P.tokenizer.pad_token_id
    label_str = processor_P.batch_decode(labels_ids, group_tokens=False)
    #wer = wer_metric.compute(predictions=pred_str, references=label_str)
    error_rate = jiwer.wer(label_str, pred_str)
    return {"wer": error_rate}

training_args = TrainingArguments(
    output_dir="./wav2vec2_finetuned",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy="steps",
    num_train_epochs=3,
    save_steps=500,
    eval_steps=10000,
    logging_steps=500,
    learning_rate=1e-5,
    warmup_steps=500,
    save_total_limit=2,
    report_to=["none"]  # 禁用 wandb
)

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(processor_P)

trainer = Trainer(
    model=model_P,
    args=training_args,
    train_dataset=dataset1,
    eval_dataset=dataset1,
    tokenizer=processor_P.feature_extractor,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

c:\Users\Alex\anaconda3\envs\BayesPCN\lib\site-packages\accelerate\accelerator.py:457: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/5040 [00:00<?, ?it/s]

{'loss': 19.6299, 'grad_norm': 320.8553161621094, 'learning_rate': 1e-05, 'epoch': 0.3}
{'loss': 15.2253, 'grad_norm': 669.8720703125, 'learning_rate': 8.898678414096917e-06, 'epoch': 0.6}
{'loss': 14.2634, 'grad_norm': 181.22914123535156, 'learning_rate': 7.797356828193832e-06, 'epoch': 0.89}
{'loss': 12.2134, 'grad_norm': 1531.861572265625, 'learning_rate': 6.69603524229075e-06, 'epoch': 1.19}


Checkpoint destination directory ./wav2vec2_finetuned\checkpoint-2500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'loss': 11.6477, 'grad_norm': 320.7424621582031, 'learning_rate': 5.594713656387666e-06, 'epoch': 1.49}


Checkpoint destination directory ./wav2vec2_finetuned\checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'loss': 10.0332, 'grad_norm': 665.748779296875, 'learning_rate': 4.493392070484582e-06, 'epoch': 1.79}
{'loss': 10.1711, 'grad_norm': 36.92812728881836, 'learning_rate': 3.3920704845814985e-06, 'epoch': 2.08}
{'loss': 10.2484, 'grad_norm': 9.26842212677002, 'learning_rate': 2.290748898678414e-06, 'epoch': 2.38}
{'loss': 9.8952, 'grad_norm': 206.91778564453125, 'learning_rate': 1.1894273127753305e-06, 'epoch': 2.68}
{'loss': 9.3833, 'grad_norm': 312.0986022949219, 'learning_rate': 8.810572687224672e-08, 'epoch': 2.98}
{'train_runtime': 552.6679, 'train_samples_per_second': 9.119, 'train_steps_per_second': 9.119, 'train_loss': 12.247346962822808, 'epoch': 3.0}


TrainOutput(global_step=5040, training_loss=12.247346962822808, metrics={'train_runtime': 552.6679, 'train_samples_per_second': 9.119, 'train_steps_per_second': 9.119, 'train_loss': 12.247346962822808, 'epoch': 3.0})

In [354]:
output_dir = r".\wav2vec2_finetuned"
checkpoints = [f.path for f in os.scandir(output_dir) if f.is_dir() and 'checkpoint' in f.name]

# 获取最新的 checkpoint
latest_checkpoint = max(checkpoints, key=os.path.getmtime)
model_tuned = Wav2Vec2ForCTC.from_pretrained(latest_checkpoint)
model_tuned.eval()

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [311]:
audio_dir1 =r".\speech_files"

audio_files1 = [f for f in os.listdir(audio_dir1) if f.endswith('.wav')]
audio_files1[0]

'ALL_016_M_CMN_ENG_HT1.wav'

In [364]:
wave.shape

(2397271,)

In [375]:
wave, sr = librosa.load(os.path.join(audio_dir1,audio_files1[0]))
wave_res = librosa.resample(wave, orig_sr=sr, target_sr=16000)
start=int(tg_sentence[2].minTime*16000)
end=int(tg_sentence[2].maxTime*16000)


input=processor_P(wave_res[start:end], sampling_rate=16000, return_tensors="pt").input_values.to(device)

model_tuned.to(device)
with torch.no_grad():
    out_encoder=model_tuned(input).logits

outind=torch.argmax(out_encoder,dim=-1).cpu().numpy()
transcription = processor_P.batch_decode(outind)[0].split(" ")

In [383]:
processor_P.batch_decode(outind)[0]

'b ɪ ɡ d ɑː ɡ z k æ n b iː d eɪ n dʒ ɚ ɹ ə s'

In [388]:
list(processor_P.tokenizer.get_vocab())[21]

'z'

In [382]:
outind

array([[ 0, 26,  0,  0, 17,  0,  0,  0,  0,  0,  0, 35,  0,  0,  0,  0,
         0, 12,  0,  0, 41,  0,  0,  0,  0,  0,  0,  0,  0, 35,  0, 21,
         0,  0,  0,  0,  0,  0, 11,  0,  0, 36,  0,  0,  4,  4,  0,  0,
        26, 30,  0,  0,  0,  0,  0, 12,  0,  0, 44,  0,  0,  0,  0,  0,
         4,  0, 60,  0,  0, 43,  0, 27,  0,  0,  7,  0,  0,  0,  0,  5,
         0,  0,  0,  0,  0,  0,  0,  0,  0]], dtype=int64)

In [415]:
def align_sequences(seq1, seq2):
    len1, len2 = len(seq1), len(seq2)
    dp = np.zeros((len1 + 1, len2 + 1), dtype=int)

    for i in range(len1 + 1):
        dp[i][0] = i
    for j in range(len2 + 1):
        dp[0][j] = j

    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            cost = 0 if seq1[i - 1] == seq2[j - 1] else 1
            dp[i][j] = min(dp[i - 1][j] + 1,   
                           dp[i][j - 1] + 1,   
                           dp[i - 1][j - 1] + cost)       
    aligned_seq1, aligned_seq2 = [], []
    i, j = len1, len2
    while i > 0 or j > 0:
        if i > 0 and dp[i][j] == dp[i - 1][j] + 1:
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append('<pad>')  
            i -= 1
        elif j > 0 and dp[i][j] == dp[i][j - 1] + 1:
            aligned_seq1.append('<pad>')
            aligned_seq2.append(seq2[j - 1])
            j -= 1
        else:
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append(seq2[j - 1])
            i -= 1
            j -= 1
    return aligned_seq1[::-1], aligned_seq2[::-1]
torch.cuda.empty_cache()
audio_files1 = [f for f in os.listdir(audio_dir1) if f.endswith('.wav')]
all_acc=[]
for each_audio in audio_files1:
    #print(each_audio)
    wave, sr = librosa.load(os.path.join(audio_dir1,each_audio))
    wave_res = librosa.resample(wave, orig_sr=sr, target_sr=16000)
    
    tg = textgrid.TextGrid.fromFile(os.path.join(audio_dir1,each_audio)[:-3]+"TextGrid")
    tg_sentence = tg[0]
    for _,i in enumerate(tg[0]):
        if i.mark!="":
            tg_sentence[_-1].maxTime=tg_sentence[_].minTime
    tg_sentence = [i for i in tg_sentence if i.mark!=""]
    
    each_acc=[]
    for each_sentence in tg_sentence:
        start_time = int(each_sentence.minTime*16000)
        end_time = int(each_sentence.maxTime*16000)
        input=processor_P(wave_res[start_time:end_time], sampling_rate=16000, return_tensors="pt").input_values.to(device)
        model_tuned.to(device)
        #print(each_sentence)
        with torch.no_grad():
            out_encoder=model_tuned(input).logits
        
        mask = np.ones(out_encoder.shape[-1], dtype=bool)
        mask[list(english_phoneme_dict.values())] = False
        out_encoder[:, :, mask] = 0
        outind=torch.argmax(out_encoder,dim=-1).cpu().numpy()
        transcription = processor_P.batch_decode(outind)[0].split(" ")
        ipalist=[arpabet_to_ipa[i.mark] for i in tg[-1] if each_sentence.minTime<=i.minTime and each_sentence.maxTime>=i.maxTime and i.mark!="" and i.mark!="sp" and i.mark!="sil"]
        #ipalist=[i.mark for i in tg[-1] if each_sentence.minTime<=i.minTime and each_sentence.maxTime>=i.maxTime and i.mark!="" and i.mark!="sp" and i.mark!="sil"]
        if ipalist==['ə', 'b', 'ɔɪ', 'f', 'ɛ', 'l', 'f', 'ɹ', 'ʌ', 'm', 'ə', 'w', 'ɪ', 'n', 'd', 'oʊ']:
            ipalist= ['ɐ', 'b', 'ɔɪ', 'f', 'ɛ', 'l', 'f', 'ɹ', 'ʌ', 'm', 'ð', 'ə', 'w', 'ɪ', 'n', 'd', 'oʊ'] 
        if len(transcription)!=len(ipalist):
            transcription, ipalist=align_sequences(transcription, ipalist)
            print(transcription,"\n", ipalist)
            
        each_acc.append(jiwer.wer(transcription, ipalist))
        #break
    all_acc.append(np.mean(each_acc))
    #break


['b', 'ɪ', 'ɡ', 'd', 'ɔ', 'ɡ', 'z', 'k', 'æ', 'n', 'b', 'ɪ', 'd', 'eɪ', 'n', 'dʒ', 'ɚ', 'ɹ', 'ə', 's'] 
 ['b', 'ɪ', 'ɡ', 'd', 'ɑ', 'ɡ', 'z', 'k', 'æ', 'n', 'b', 'i', 'd', 'eɪ', 'n', 'dʒ', 'ɚ', '<pad>', 'ə', 's']
['ð', 'ə', 'f', 'aɪ', '<pad>', 'w', 'ʌ', 'z', 'v', 'ɛ', 'ɹ', 'i', 'h', 'ɔ', 't'] 
 ['ð', 'ə', 'f', 'aɪ', 'ɚ', 'w', 'ə', 'z', 'v', 'ɛ', 'ɹ', 'i', 'h', 'ɑ', 't']
['ʃ', 'ɪ', 'ɪ', 'z', 'd', 'ɹ', 'ɪ', 'ŋ', 'k', 'ɪ', 'ŋ', 'f', 'ɹ', 'ʌ', 'm', 'h', 'ɚ', 'ɹ', 'oʊ', 'n', 'k', 'ʌ', 'p'] 
 ['ʃ', 'i', '<pad>', 'z', 'd', 'ɹ', 'ɪ', 'ŋ', 'k', 'ə', 'n', 'f', 'ɹ', 'ʌ', 'm', 'h', 'ɚ', '<pad>', 'oʊ', 'n', 'k', 'ʌ', 'p']
['ð', 'ə', 't', 'aʊ', 'ɹ', 'ɚ', 'f', 'ɛ', 'l', 'ɔ', 'n', '<pad>', 'ə', 'f', 'l', 'oʊ', '<pad>'] 
 ['ð', 'ə', 't', 'aʊ', 'ə', 'l', 'f', 'ɛ', 'l', 'ɑ', 'n', 'ð', 'ə', 'f', 'l', 'ɔ', 'ɹ']
['ð', 'ə', 'f', 'æ', 'm', 'ɪ', 'l', 'i', 'l', 'aɪ', 'k', 's', 'f', 'ɪ', 'ʃ'] 
 ['ð', 'ə', 'f', 'æ', 'm', '<pad>', 'l', 'i', 'l', 'aɪ', 'k', 's', 'f', 'ɪ', 'ʃ']
['ð', 'ə', 'b', 'ə', 'n', 'æ', 'n', 'oʊ

In [418]:
torch.cuda.empty_cache()

In [416]:
np.mean(all_acc)

0.15462063555033828

In [408]:
ipalist

['AH0',
 'B',
 'OY1',
 'F',
 'EH1',
 'L',
 'F',
 'R',
 'AH1',
 'M',
 'AH0',
 'W',
 'IH1',
 'N',
 'D',
 'OW0']

In [ ]:
for audio_file, label_file in zip(audio_files, label_files):
    # 加载音频文件
    audio_path = os.path.join(audio_dir, audio_file)
    audio, sr = librosa.load(audio_path)
    wave_res = librosa.resample(audio, orig_sr=sr, target_sr=16000)

    # 加载标注文件
    label_path = os.path.join(audio_dir, label_file)
    tg = textgrid.TextGrid.fromFile(label_path)
    tg_sentence = tg[0]
    for _,i in enumerate(tg[0]):
        if i.mark!="":
            tg_sentence[_-1].maxTime=tg_sentence[_].minTime
    tg_sentence = [i for i in tg_sentence if i.mark!=""]
    #tg_sentence = [each for _,each in enumerate(tg_sentence) if _ in set_list]


    # 切割音频文件
    for each_sentence in tg_sentence:
        start_time = each_sentence.minTime
        end_time = each_sentence.maxTime
        marks=[]
        clip = split_audio(wave_res, start_time, end_time, 16000)
        #ipalist=[arpabet_to_ipa[i.mark] for i in tg[-1] if tg_sentence[0].minTime<=i.minTime and tg_sentence[0].maxTime>=i.maxTime and i.mark!="" and i.mark!="sp" and i.mark!="sil"]
        #audio_clips.append(clip)
        finetunedata["audio"].append(clip)
        #labels.append(" ".join(ipalist))#np.array([[english_phoneme_dict[i] for i in ipalist]])
        finetunedata["transcription"].append(each_sentence.mark)#[english_phoneme_dict[i] for i in ipalist])

In [381]:
from pydub import AudioSegment
speech=AudioSegment.from_wav(os.path.join(audio_dir1,audio_files1[0]))
speech[tg_sentence[2].minTime*1000:tg_sentence[2].maxTime*1000]

In [376]:
transcription

['b',
 'ɪ',
 'ɡ',
 'd',
 'ɑː',
 'ɡ',
 'z',
 'k',
 'æ',
 'n',
 'b',
 'iː',
 'd',
 'eɪ',
 'n',
 'dʒ',
 'ɚ',
 'ɹ',
 'ə',
 's']

In [360]:
tg = textgrid.TextGrid.fromFile(os.path.join(audio_dir1,audio_files1[0])[:-3]+"TextGrid")
tg_sentence = tg[0]
for _,i in enumerate(tg[0]):
    if i.mark!="":
        tg_sentence[_-1].maxTime=tg_sentence[_].minTime
tg_sentence = [i for i in tg_sentence if i.mark!=""]

[Interval(0.042, 1.552, A BOY FELL FROM A WINDOW),
 Interval(1.774, 3.604, THE WIFE HELPED HER HUSBAND),
 Interval(3.704, 5.495, BIG DOGS CAN BE DANGEROUS),
 Interval(5.582, 7.221, THE SHOES WERE VERY DIRTY),
 Interval(7.305, 8.925, THE PLAYER LOST A SHOE),
 Interval(9.065, 10.445, SOMEBODY STOLE THE MONEY),
 Interval(10.549, 12.119, THE FIRE WAS VERY HOT),
 Interval(12.458, 14.249, SHE'S DRINKING FROM HER OWN CUP),
 Interval(14.555, 16.115, THE PICTURE CAME FROM A BOOK),
 Interval(16.404, 18.224, THE CAR IS GOING TOO FAST),
 Interval(18.363, 20.173, THE PAINT DRIPPED ON THE GROUND),
 Interval(20.267, 21.986, THE TOWEL FELL ON THE FLOOR),
 Interval(22.111, 23.661, THE FAMILY LIKES FISH),
 Interval(23.746, 25.415, THE BANANAS ARE TOO RIPE),
 Interval(25.774, 27.634, HE GREW LOTS OF VEGETABLES),
 Interval(27.751, 29.62, SHE ARGUES WITH HER SISTER),
 Interval(29.709, 31.509, THE KITCHEN WINDOW WAS CLEAN),
 Interval(31.631, 33.36, HE HUNG UP HIS RAINCOAT),
 Interval(33.468, 35.059, THE MAI

In [366]:
processor_P.batch_decode(outind)[0]

't h ɜː h ʌ z b ə n d b iː k'